

## Reference

https://pytorch.org/tutorials/beginner/data_loading_tutorial.html


## Library imports

In [ ]:
# common imports
import os
import random
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
#import math
#import time
#from skimage import io, transform
#from typing import Dict
#from pathlib import Path

# interactive plot libraries
import matplotlib.pyplot as plt
import seaborn as sns
#from plotly.offline import init_notebook_mode, iplot # download_plotlyjs, plot
#import plotly.graph_objs as go
#from plotly.subplots import make_subplots
#init_notebook_mode(connected=True)

# torch imports
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from torchvision.models.resnet import resnet50, resnet18, resnet34, resnet101
import torch.nn.functional as F

## Config files

In [ ]:
cfg = {
    'train_img_path': "cassava-leaf-disease-classification/train_images/",
    'train_csv_path': 'cassava-leaf-disease-classification/train.csv',
    
    'model_params': {
        'model_architecture': 'resnet18', 'model_name': "R18_pretrain_imagenet",
        'lr': 1e-4, 'weight_path': "", 
        'lr_find' : 0, 'train': 1, 'validate': 0,'test': 0 },

    'train_data_loader': { 'batch_size': 16, 'shuffle': True, 'num_workers': 4 },
    
    'val_data_loader': {'batch_size': 16, 'shuffle': True, 'num_workers': 4 },

    'test_data_loader': {'batch_size': 32, 'shuffle': False, 'num_workers': 4 },

    'train_params': {'train_start_batch_index' : 117001, 'max_num_steps': 11, 'checkpoint_every_n_steps': 5 } }

In [ ]:
index_label_map = {
                0: "Cassava Bacterial Blight (CBB)", 
                1: "Cassava Brown Streak Disease (CBSD)",
                2: "Cassava Green Mottle (CGM)", 
                3: "Cassava Mosaic Disease (CMD)", 
                4: "Healthy"
                }

## EDA

In [ ]:
#train_csv = pd.read_csv('cassava-leaf-disease-classification/train.csv')
#train_csv['disease'] = train_csv['label'].map(index_label_map);
#print(train_csv.shape)
#train_csv.head()

In [ ]:
#_, axes = plt.subplots(1, 1, figsize=(16, 8))
#sns.countplot(x='disease', data=train_csv, ax=axes);

In [ ]:
#print(train_csv['disease'].value_counts(normalize=True))

## TODO

1. load images into dataset (Dataset class of pytorch maybe)
2. split into 5 fold data - scikit learn
3. simple network -r18, r50 with last layers changed to 5 lables
4. adam optimizer, lr_finder, cross entropy loss
5. cv score

## Helper functions

In [ ]:
def find_no_of_trainable_params(model):
    total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    #print(total_trainable_params)
    return total_trainable_params

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
set_seed(42)

## Dataset class

In [ ]:
class CassavaDataset(Dataset):
    """Cassave leaf disease detection dataset."""

    def __init__(self, csv_file, root_dir, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.cassava_leaf_disease = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.cassava_leaf_disease)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.cassava_leaf_disease.iloc[idx, 0])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        
        label = np.array(self.cassava_leaf_disease.iloc[idx, 1])
        return (image, label)

## Transforms and Dataloader

In [ ]:
train_transform = transforms.Compose([transforms.Resize(255),
                                transforms.CenterCrop(224),
                                transforms.ToTensor()])

cassava_train_dataset = CassavaDataset( csv_file=cfg['train_csv_path'],
                                        root_dir=cfg['train_img_path'],
                                        transform=train_transform)

trainloader = DataLoader(cassava_train_dataset,
                         batch_size=cfg['train_data_loader']['batch_size'], 
                         shuffle=cfg['train_data_loader']['shuffle'])

## Pretrained model

In [ ]:
model = models.resnet34(pretrained=True)
print('1 : ', find_no_of_trainable_params(model))
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
print('2 : ', find_no_of_trainable_params(model))
fc_layer = nn.Sequential(nn.Linear(in_features=512, out_features=128), nn.ReLU(),
                         nn.Linear(in_features=128, out_features=5))
                        #nn.LogSoftmax(dim=1))
model.fc = fc_layer
print('3 : ', find_no_of_trainable_params(model))
#print(model)

## Device, loss fn, optimizer

In [ ]:
# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device);

# loss function
criterion = nn.CrossEntropyLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

## Training loop

In [ ]:
epochs = 1
running_loss = 0
tr_it = iter(trainloader)
progress_bar = tqdm(range(len(trainloader)))

for epoch in range(epochs):
    for i in progress_bar:
        try:
            inputs, labels = next(tr_it)
        except StopIteration:
            tr_it = iter(trainloader)
            inputs, labels = next(tr_it)
            
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        model.train()
        torch.set_grad_enabled(True)
        logits = model(inputs)
        loss = criterion(logits, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss +=loss.item()
    
        # print info
        progress_bar.set_description(f"loss: {loss.item()} loss(avg): {running_loss / (i + 1)}")